In [1]:
import json
from os.path import join
import os
os.chdir("/home/tiagoalmeida/bioASQ-taskb/")

import sys
import pickle
import gc
import numpy as np
import time

##add keras to the modules
module_path = os.path.abspath(os.path.join('pubmed_data'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pubmed_data import pubmed_helper as ph

from models.generic_model import ModelAPI, f_recall, f_map



import


### Load training and test data

In [2]:
bioASQ_data_path = "/backup/BioASQ-training7b/"
bioASQ_data_train = json.load(open(os.path.join(bioASQ_data_path,"7b_train_split.json")))
bioASQ_data_test = json.load(open(os.path.join(bioASQ_data_path,"7b_test_split.json")))

#Manual add entry to the training data because they dont have any relevant document
bioASQ_data_train[1819]["documents"] = ["none"]
bioASQ_data_train[1836]["documents"] = ["none"]

#goldstandard
bioASQ_test_expectation = list(map(lambda x:x["documents"],bioASQ_data_test))

#load pmid_index mapping
pmid_index_mapping = ph.pmid_index_mapping()

Load /backup/saved_models/pmid_index_mapping.p


# Index of tests

 - [Baseline BM25](#bm25)
 - [AWE](#awe)
 - [AWE-TFIDF](#awe_tfidf)
 - [DSSM](#dssm_tribag_same_q_doc)
 - [DEEP RANK](#deep_rank)

<a id='bm25'></a>

### Manual check to the bm25 results since they are saved in memory

In [4]:
### load the train set!
LOAD_TRAIN = False

path="/backup/results/bm25"
files = sorted(filter(lambda x: "test" in x and "phase" not in x,os.listdir(path)))
print("Test files:",files)
bm25_test_results = []
for file in files:
    with open(os.path.join(path,file),"rb") as f:
        bm25_test_results.extend(pickle.load(f))
if LOAD_TRAIN:
    files = sorted(filter(lambda x: "train" in x,os.listdir(path)))
    print("Train files:",files)
    bm25_train_results = []
    for file in files:
        with open(os.path.join(path,file),"rb") as f:
            for r in pickle.load(f):
                bm25_train_results.append(r[:5000])


Test files: ['results_test_0.p', 'results_test_1.p']


In [5]:
print("TEST set, len predictions",len(bm25_test_results),"len expecations",len(bioASQ_data_test))

expectations = list(map(lambda x:x["documents"],bioASQ_data_test))

print("Recall@1000:",f_recall(bm25_test_results,expectations,at=1000))
print("Recall@2500:",f_recall(bm25_test_results,expectations,at=2500))
print("Recall@5000:",f_recall(bm25_test_results,expectations,at=5000))
print("MAP:",f_map(bm25_test_results,expectations))
print("MAP bioASQ:",f_map(bm25_test_results,expectations,bioASQ=True))
#print("MAP with fill the rest:",f_map(bm25_test_results,expectations,fill_rest=True))

TEST set, len predictions 549 len expecations 549
Recall@1000: 0.8323096260300467
Recall@2500: 0.8746784560651255
Recall@5000: 0.908476240984081
MAP: 0.48321889652778066
MAP bioASQ: 0.15353167375025284


In [13]:
print("TRAIN set, len predictions",len(bm25_train_results),"len expecations",len(bioASQ_data_train))

expectations = list(map(lambda x:x["documents"],bioASQ_data_train))

print("Recall:",f_recall(bm25_train_results,expectations,at=5000))
print("MAP:",f_map(bm25_train_results,expectations))
print("MAP bioASQ:",f_map(bm25_train_results,expectations,bioASQ=True))

TRAIN set, len predictions 2198 len expecations 549
Recall: 0.9151476322586187
MAP: 0.48115157297880273
MAP bioASQ: 0.15416704579921153


<a id='awe'></a>
## Evaluate AWE

In [3]:
from models.generic_model import AWE

#MODE = "regex_full_tokens"
MODE = "regex_less_700k_freq"

#Load
tk = ph.load_tokenizer(MODE)
model = AWE.load(MODE+"_title_abs")

model.num_docs = 18824355

Load regex_less_700k_freq_tokenizer.p


In [4]:
#Test data
bioASQ_test_queries_tokenized = tk.texts_to_sequences(map(lambda x:x["body"],bioASQ_data_test))
bioASQ_test_expectation = list(map(lambda x:x["documents"],bioASQ_data_test))

In [1]:
import time

model.top_k=10000
start_time = time.time()
model.metrics = [f_recall]
_predictions = model.evaluate(batch_size=50,
               expectations=bioASQ_test_expectation,
               queries=bioASQ_test_queries_tokenized,
               index_pmid_mapping = pmid_index_mapping.inverse,
                at=[1000,2500,5000])

print("Overall time", time.time()-start_time)

print(_predictions)

NameError: name 'model' is not defined

<a id='awe_tfidf'></a>
## Evaluate AWE TFIDF

In [3]:
from models.generic_model import AWE_TFIDF


#MODE = "regex_full_tokens"
MODE = "regex_less_700k_freq"

#Load
tk = ph.load_tokenizer(MODE)
model = AWE_TFIDF.load(MODE+"_title_abs")

model.num_docs = 18824355

Load regex_less_700k_freq_tokenizer.p


In [4]:
#Test data

bioASQ_test_queries_tokenized = tk.texts_to_sequences(map(lambda x:x["body"],bioASQ_data_test))
bioASQ_test_expectation = list(map(lambda x:x["documents"],bioASQ_data_test))

In [5]:
import time

model.top_k=10000
start_time = time.time()
model.metrics = [f_recall]
_predictions = model.evaluate(batch_size=50,
               expectations=bioASQ_test_expectation,
               queries=bioASQ_test_queries_tokenized,
               index_pmid_mapping = pmid_index_mapping.inverse,
                at=[1000,2500,5000])

print("Overall time", time.time()-start_time)


DEBUG: doc_awe_emb shape (18824355, 200)
DEBUG: queries_awe_emb transpose shape (200, 50)
DEBUG: doc_norm shape (18824355, 1)
DEBUG: queries_norm shape (1, 50)
DEBUG: norm shape (18824355, 50)
DEBUG: dot_product shape (18824355, 50)
predict the top 10000
Sorting time: 38.72075438499451
DEBUG: doc_awe_emb shape (18824355, 200)
DEBUG: queries_awe_emb transpose shape (200, 50)
DEBUG: doc_norm shape (18824355, 1)
DEBUG: queries_norm shape (1, 50)
DEBUG: norm shape (18824355, 50)
DEBUG: dot_product shape (18824355, 50)
predict the top 10000
Sorting time: 38.33282494544983
DEBUG: doc_awe_emb shape (18824355, 200)
DEBUG: queries_awe_emb transpose shape (200, 50)
DEBUG: doc_norm shape (18824355, 1)
DEBUG: queries_norm shape (1, 50)
DEBUG: norm shape (18824355, 50)
DEBUG: dot_product shape (18824355, 50)
predict the top 10000
Sorting time: 39.107759952545166
DEBUG: doc_awe_emb shape (18824355, 200)
DEBUG: queries_awe_emb transpose shape (200, 50)
DEBUG: doc_norm shape (18824355, 1)
DEBUG: queri

## Test average weight embedding

In [3]:
from models.generic_model import AWE, AWE_TFIDF

#MODE = "regex_full_tokens"
MODE = "regex_less_700k_freq"

#Load
tk = ph.load_tokenizer(MODE)
model = AWE_TFIDF.load(MODE+"_title_abs")

model.num_docs = 18824355

Load regex_less_700k_freq_tokenizer.p


In [4]:


bioASQ_test_queries_tokenized = tk.texts_to_sequences(map(lambda x:x["body"],bioASQ_data_test))
bioASQ_test_expectation = list(map(lambda x:x["documents"],bioASQ_data_test))



Load /backup/saved_models/pmid_index_mapping.p


In [5]:
predictions = model.predict(bioASQ_test_queries_tokenized[:2]).T

Invalid query 0
Invalid query 1
DEBUG: doc_awe_emb shape (18824355, 200)
DEBUG: queries_awe_emb transpose shape (200, 2)
DEBUG: doc_norm shape (18824355, 1)
DEBUG: queries_norm shape (1, 2)
DEBUG: norm shape (18824355, 2)
DEBUG: dot_product shape (18824355, 2)


In [5]:
model.evaluate(batch_size=32,
               expectations=bioASQ_test_expectation,
               queries=bioASQ_test_queries_tokenized,
               index_pmid_mapping = pmid_index_mapping.inverse)

DEBUG: doc_awe_emb shape (18824355, 200)
DEBUG: queries_awe_emb transpose shape (200, 32)
DEBUG: doc_norm shape (18824355, 1)
DEBUG: queries_norm shape (1, 32)
DEBUG: norm shape (18824355, 32)
DEBUG: dot_product shape (18824355, 32)
DEBUG: doc_awe_emb shape (18824355, 200)
DEBUG: queries_awe_emb transpose shape (200, 32)
DEBUG: doc_norm shape (18824355, 1)
DEBUG: queries_norm shape (1, 32)
DEBUG: norm shape (18824355, 32)
DEBUG: dot_product shape (18824355, 32)
DEBUG: doc_awe_emb shape (18824355, 200)
DEBUG: queries_awe_emb transpose shape (200, 32)
DEBUG: doc_norm shape (18824355, 1)
DEBUG: queries_norm shape (1, 32)
DEBUG: norm shape (18824355, 32)
DEBUG: dot_product shape (18824355, 32)
DEBUG: doc_awe_emb shape (18824355, 200)
DEBUG: queries_awe_emb transpose shape (200, 32)
DEBUG: doc_norm shape (18824355, 1)
DEBUG: queries_norm shape (1, 32)
DEBUG: norm shape (18824355, 32)
DEBUG: dot_product shape (18824355, 32)
DEBUG: doc_awe_emb shape (18824355, 200)
DEBUG: queries_awe_emb tran

[('f_map', 0.0376222280047417), ('f_recall', 0.42359242654430795)]

<a id='dssm_tribag_same_q_doc'></a>

## TEST DSSM

In [3]:
from models.fast_neural_retrieval.dssm_model import DSSM, TRIGRAM_VOC



bioASQ_test_queries_body = list(map(lambda x:x["body"],bioASQ_data_test))
bioASQ_test_expectation = list(map(lambda x:x["documents"],bioASQ_data_test))


Load hashtrick_full_tokens_tokenizer.p


In [4]:
dssm = DSSM.load("trigram_same_q_d_4neg")

In [5]:
import time
N=[1000,2500,5000]


dssm.top_k=10000
start_time = time.time()

_predictitons=dssm.evaluate( batch_size=50,
               expectations=bioASQ_test_expectation,
               queries=bioASQ_test_queries_body,
               index_pmid_mapping = pmid_index_mapping.inverse,
               at = N)

print("Overall time", time.time()-start_time)



50/50 [==============================] - 0s 1ms/step
DEBUG: doc_awe_emb shape (18824355, 128)
DEBUG: queries_awe_emb transpose shape (128, 50)
DEBUG: doc_norm shape (18824355, 1)
DEBUG: queries_norm shape (1, 50)
DEBUG: norm shape (18824355, 50)
DEBUG: dot_product shape (18824355, 50)
predict the top 10000
Sorting time: 30.766916036605835
50/50 [==============================] - 0s 502us/step
DEBUG: doc_awe_emb shape (18824355, 128)
DEBUG: queries_awe_emb transpose shape (128, 50)
DEBUG: doc_norm shape (18824355, 1)
DEBUG: queries_norm shape (1, 50)
DEBUG: norm shape (18824355, 50)
DEBUG: dot_product shape (18824355, 50)
predict the top 10000
Sorting time: 30.83276891708374
50/50 [==============================] - 0s 493us/step
DEBUG: doc_awe_emb shape (18824355, 128)
DEBUG: queries_awe_emb transpose shape (128, 50)
DEBUG: doc_norm shape (18824355, 1)
DEBUG: queries_norm shape (1, 50)
DEBUG: norm shape (18824355, 50)
DEBUG: dot_product shape (18824355, 50)
predict the top 10000
Sorting

AttributeError: 'list' object has no attribute 'shape'

'How could iPSCs be used for the treatment of diabetes?'

In [5]:
dssm.predict(bioASQ_test_queries_body)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0,...

In [218]:

p = np.argpartition(-r, 100000, axis=0)[:100000,:]

In [219]:
p

array([[17337430,  2217453, 17670079, 18488415, 17005652],
       [ 5295701,  4896649, 12274923, 13727632, 16476832],
       [ 4645437,  1808511, 18251732,  1264568,  8247162],
       ...,
       [17154372,  2022710,  8500257,    56147,   560190],
       [14087007,   425006, 17832560,  2478349, 12638176],
       [12132492,  9274190,  2454101,  3105630,  2062167]])

In [220]:
reshape = r[p, np.arange(p.shape[1])]

argsort = np.argsort(-reshape,axis=0)
#reshape = -np.sort(-reshape, axis=0)

p[argsort, np.arange(argsort.shape[1])]

In [203]:
argsort

array([[33922, 59367, 22099, 51320, 46795],
       [ 4597, 61952, 13826, 17136, 14484],
       [27039, 55488, 12731, 57748, 10858],
       ...,
       [99998, 99997, 99998, 99998, 99997],
       [99994, 99998, 99996, 99997, 99998],
       [99995, 99999, 99999, 99999, 99999]])

In [209]:
p[argsort, np.arange(argsort.shape[1])]

array([[11040725, 13593453, 17970655, 15846496, 15332412],
       [  329620,  4942267,  7327770, 10084591,  3975174],
       [18723874,   618373,  3086913, 11621512, 14568144],
       ...,
       [14087007,  2022710, 17832560,  2478349,   560190],
       [13976516,   425006,  9152679,    56147, 12638176],
       [ 8177325,  9274190,  2454101,  3105630,  2062167]])

In [215]:
r[8177325,0]

0.9444565105724636

In [217]:
r

array([[0.88081825, 0.87613495, 0.87151676, 0.89276173, 0.81761281],
       [0.90585468, 0.90718728, 0.89120494, 0.90733109, 0.85100793],
       [0.91669974, 0.89598432, 0.91167512, 0.91110668, 0.83358249],
       ...,
       [0.91819576, 0.90993734, 0.92661172, 0.89442718, 0.85032848],
       [0.91305674, 0.90290071, 0.90147887, 0.8897185 , 0.84382922],
       [0.92485938, 0.90326949, 0.92039408, 0.91274687, 0.84756293]])

In [162]:
a = np.array([[1,2,3], [4,5,6], [7,8,9]])
a[idx, np.arange(idx.shape[1])]


array([[4, 2, 9],
       [1, 5, 3]])

In [159]:
idx = np.array([[1,0],[0,1],[2,0]])
idx = idx.T

In [163]:
a

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [160]:
idx

array([[1, 0, 2],
       [0, 1, 0]])

In [158]:
a[:,0]

array([1, 4, 7])

In [3]:
d.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dssm_query_input (InputLayer)   (None, 48482)        0                                            
__________________________________________________________________________________________________
dssm_pos_doc_input (InputLayer) (None, 48482)        0                                            
__________________________________________________________________________________________________
dssm_neg_doc_input_0 (InputLaye (None, 48482)        0                                            
__________________________________________________________________________________________________
dssm_neg_doc_input_1 (InputLaye (None, 48482)        0                                            
__________________________________________________________________________________________________
dssm_neg_d

In [4]:
d.get_layer("siamese_model").get_layer("projection").summary()

AttributeError: 'DSSM_Projection' object has no attribute 'summary'

In [5]:
#d = load_model("/backup/saved_models/same_q_d_model_dssm_model.h5")
d.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dssm_query_input (InputLayer)   (None, 48482)        0                                            
__________________________________________________________________________________________________
dssm_pos_doc_input (InputLayer) (None, 48482)        0                                            
__________________________________________________________________________________________________
dssm_neg_doc_input_0 (InputLaye (None, 48482)        0                                            
__________________________________________________________________________________________________
dssm_neg_doc_input_1 (InputLaye (None, 48482)        0                                            
__________________________________________________________________________________________________
dssm_neg_d

In [8]:
from models.fast_neural_retrieval.dssm_model import dssm_projectiom_model
from tensorflow.keras.layers import Input

In [11]:
f = dssm_projectiom_model("cenas")

In [9]:
i = Input(shape=(None,48482))

In [13]:
f(inputs=i)

TypeError: build_model() missing 1 required positional argument: 'self'

<a id='bm25_deep_rank'></a>

## BM25 data to deep RANK

In [6]:
import tarfile

path_dl_test = "/backup/results/fast_method_relevant_results/train_data_deep_models_v2.tar.gz"

tar = tarfile.open(path_dl_test)
#open
print("Open",path_dl_test)
m = tar.getmembers()[0]
f = tar.extractfile(m)
test_articles_collection = pickle.load(f)

f.close()

Open /backup/results/fast_method_relevant_results/train_data_deep_models_v2.tar.gz


KeyboardInterrupt: 

In [ ]:
path_file="/backup/results/deep_rank/deep_rank_v2_train_data.p"

with open(path_file,"rb") as f:
    deep_rank_test_query_results = pickle.load(f)

bio_docs = dict(map(lambda x:(x["id"],{"bm25_result":x["documents"],"goldstandard":x["positive_pmid"]}),test_articles_collection["bioasq_data"]))    

id_to_remove = []
for k,v in deep_rank_test_query_results.items():
    
    if len(v["goldstandard"]) == 0:
        id_to_remove.append(k)
        continue
    
    bio_docs[k]["goldstandard"] = v["goldstandard"]

for k in id_to_remove:
    del bio_docs[k]
    
bio_docs = list(bio_docs.values())

print("TEST set, len ",len(bio_docs))

expectations = list(map(lambda x:x["goldstandard"],bio_docs))
predictions = list(map(lambda x:x["bm25_result"],bio_docs))

print("Recall:",f_recall(predictions,expectations,at=1000))
print("MAP:",f_map(predictions,expectations, use_len=True))
print("MAP @50:",f_map(predictions,expectations, bioASQ=True, at=50))
print("MAP @100:",f_map(predictions,expectations, bioASQ=True, at=100))
print("MAP @200:",f_map(predictions,expectations, bioASQ=True, at=200))
print("MAP @300:",f_map(predictions,expectations, bioASQ=True, at=300))
print("MAP bioASQ:",f_map(predictions,expectations,bioASQ=True))

<a id='deep_rank'></a>

## DEEP RANK

In [26]:
path_file="/backup/results/deep_rank/deep_rank_v2_17_1_test_data.p"

with open(path_file,"rb") as f:
    deep_rank_test_query_results = pickle.load(f)

In [27]:
id_to_remove = []
for k,v in deep_rank_test_query_results.items():
    if len(v["goldstandard"]) == 0:
        id_to_remove.append(k)
        
for k in id_to_remove:
    del deep_rank_test_query_results[k]

deep_rank_test_query_results = list(deep_rank_test_query_results.values())

print("TEST set, len ",len(deep_rank_test_query_results))

expectations = list(map(lambda x:x["goldstandard"],deep_rank_test_query_results))
predictions = list(map(lambda x:x["result"],deep_rank_test_query_results))

#print("Recall:",f_recall(predictions,expectations,at=1000))
print("MAP @10 bioASQ:",f_map(predictions,expectations,bioASQ=True))
print("MAP @25:",f_map(predictions,expectations, bioASQ=True, at=25))
print("MAP @50:",f_map(predictions,expectations, bioASQ=True, at=50))
print("MAP @100:",f_map(predictions,expectations, bioASQ=True, at=100))
print("MAP @200:",f_map(predictions,expectations, bioASQ=True, at=200))
print("MAP @300:",f_map(predictions,expectations, bioASQ=True, at=300))

print("MAP:",f_map(predictions,expectations, use_len=True))

TEST set, len  489
MAP @10 bioASQ: 0.16640820268120873
MAP @25: 0.2418029506259341
MAP @50: 0.2794877296558311
MAP @100: 0.3006927072390499
MAP @200: 0.3106717405497745
MAP @300: 0.31393638528831924
MAP: 0.2903386468781293


In [28]:
path_file="/backup/results/deep_rank/deep_rank_v2_train_data.p"

with open(path_file,"rb") as f:
    deep_rank_train_set = pickle.load(f)

In [36]:

print("TRAIN set, len predictions",len(deep_rank_train_set),"len expecations",len(bioASQ_data_train))
expectations = []
deep_rank_train_results = []
tested_id = []

#deep_rank_train_set = list(map(lambda x:x["result"],bioASQ_data_train))
expectations = list(map(lambda x:x["documents"],bioASQ_data_train))

for i,bioasq_query in enumerate(bioASQ_data_train):
    if bioasq_query["id"] in deep_rank_train_set:
        tested_id.append(bioasq_query["id"])
        deep_rank_train_results.append(deep_rank_train_set[bioasq_query["id"]]["result"])
    else:
        deep_rank_train_results.append(bm25_train_results[i])


print("TRAIN set, len predictions",len(deep_rank_train_results),"len expecations",len(expectations))

print("Recall:",f_recall(deep_rank_train_results,expectations,at=5000))
print("MAP:",f_map(deep_rank_train_results,expectations))
print("MAP @100:",f_map(deep_rank_train_results,expectations, at=100))
print("MAP bioASQ:",f_map(deep_rank_train_results,expectations,bioASQ=True))

TRAIN set, len predictions 2137 len expecations 2198
TRAIN set, len predictions 2198 len expecations 2198
Recall: 0.8503975538109899
MAP: 0.513108553800485
MAP @100: 0.4198523252766251
MAP bioASQ: 0.17122533833066206
